In [2]:
import sympy as sy
import numpy as np
import matplotlib.pyplot as plt

nondef1, nondef2, nondef3, nondef4, nondef5 = sy.symbols('nondef1, nondef2, nondef3, nondef4, nondef5')      

In [3]:
# Präfixe
_nano = 10 ** (-9)
_mikro = 10 ** (-6)
_milli = 10 ** (-3)
_centi = 10 ** (-2)
_dezi = 10 ** (-1)

_Kilo = 10 ** 3
_Mega = 10 ** 6
_Giga = 10 ** 9
_Tera = 10 ** 12

# Winkel
_degree = 2*np.pi/360

In [4]:
# feste Parameter und ihre Zahlenwerte: diese werden später in der Fehlerfunktion numerisch gemacht aber bleiben für die partiellen Ableitungen symbolisch
g, pi, mu_0, G = sy.symbols('g, pi, mu_0, G')            
                                                
fix_parameters = {
    G: 6.672 * 10 ** (-11),
    g: 9.81, 
    pi: 3.141,
    mu_0: 4 * np.pi * 10 **(-7),
}   

In [5]:
# verwendbare, symbolische Variablen, Zusätzliche einfach hinzufügen
t, U0_sy, U90_sy, U0_ref_sy, U90_ref_sy, a, n1, n2, Dn, Sn, lam1, lam2, Bn = sy.symbols('t, U0_sy, U90_sy, U0_ref_sy, U90_ref_sy, a, n1, n2, Dn, Sn, lam1, lam2, Bn')                     

In [6]:
def err_gauss(f, var1, x1, err1, var2=nondef1, x2=0, err2=0, var3=nondef2, x3=0, err3=0, var4=nondef3, x4=0, err4=0, var5=nondef4, x5=0, err5=0):
    f_x1 = sy.diff(f, var1)
    # print(f'Ableitung nach {var1}: {f_x1}')   
    f_x1 = f_x1.subs({var2: x2, var3: x3, var4: x4, var5: x5})
    f_x2 = sy.diff(f, var2)
    # if f_x2 != 0:
        # print(f'Ableitung nach {var2}: {f_x2}')   
    f_x2 = f_x2.subs({var1: x1, var3: x3, var4: x4, var5: x5})
    f_x3 = sy.diff(f, var3)
    # if f_x3 != 0:
        # print(f'Ableitung nach {var3}: {f_x3}')   
    f_x3 = f_x3.subs({var1: x1, var2: x2, var4: x4, var5: x5})
    f_x4 = sy.diff(f, var4)
    # if f_x4 != 0:
        # print(f'Ableitung nach {var4}: {f_x4}')   
    f_x4 = f_x4.subs({var1: x1, var2: x2, var3: x3, var5: x5})
    f_x5 = sy.diff(f, var5)
    # if f_x5 != 0:
        # print(f'Ableitung nach {var5}: {f_x5}')   
    f_x5 = f_x5.subs({var1: x1, var2: x2, var3: x3, var4: x4})

    # feste paramter numerisch machen
    f = f.subs(fix_parameters)
    f_x1 = f_x1.subs(fix_parameters)
    f_x2 = f_x2.subs(fix_parameters)
    f_x3 = f_x3.subs(fix_parameters)
    f_x4 = f_x4.subs(fix_parameters)
    f_x5 = f_x5.subs(fix_parameters)

    value = f.subs({var1: x1, var2: x2, var3: x3, var4: x4, var5: x5})
    error = (err1**2*(f_x1.subs(var1, x1))**2+err2**2*(f_x2.subs(var2, x2))**2+err3**2*(f_x3.subs(var3, x3))**2+err4**2*(f_x4.subs(var4, x4))**2+err5**2*(f_x5.subs(var5, x5))**2)**0.5
    # print(f'Wert: {value} +/- Fehler: {error}')
    return (value, error)

In [7]:
def gew_mittel(ar_val, ar_err):
    value_num = 0
    sigma_denom = 0
    for i in range(len(ar_val)):
        value_num += ar_val[i]/ar_err[i]**2
        sigma_denom += 1/ar_err[i]**2
    value = value_num/sigma_denom
    sigma = (1/sigma_denom)**0.5
    print(f'gew. Mittelwert: {value} +/- sigma: {sigma}')
    return (value, sigma)

In [8]:
# data Prisma

gamma = np.pi/3

angle_err = 0.05*np.pi/180
angle_green = (48+(21/30))*np.pi/180
angle_yellow = (48.5+(1/3))*np.pi/180

In [9]:
# Bestimmung Brechungsindex Prisma

n_green, n_green_err = err_gauss(
    sy.sin(a)/sy.sin(gamma/2),
    a, angle_green, angle_err
)
n_yellow, n_yellow_err = err_gauss(
    sy.sin(a)/sy.sin(gamma/2),
    a, angle_yellow, angle_err
)
print(f'n_green = {n_green} +- {n_green_err}')
print(f'n_yellow = {n_yellow} +- {n_yellow_err}')

n_green = 1.50252826700702 +- 0.00115192021745727
n_yellow = 1.50559598307150 +- 0.00114886579258733


In [10]:
# Bestimmung Dispersion Prisma

lam_green = 576.96 * _nano
lam_yellow = 546.07 * _nano

D, D_err = err_gauss(
    (n2-n1)/(lam_green-lam_yellow),
    n1, n_green, n_green_err,
    n2, n_yellow, n_yellow_err
)

print(f'D = {D} +- {D_err}')

D = 99310.9765125960 +- 52667.6213464876


In [18]:
# Bestimmung des Auflösevermögens Prisma

S, S_err = ((2.14+0.20) * _milli, (0.005*(2)**0.5) * _milli)
B_max = 33 * _milli

A, A_err = err_gauss(
    Dn*Sn/sy.cos(a),
    Sn, S, S_err,
    Dn, D, D_err,
    a, angle_yellow, angle_err
)

A_max, A_max_err = err_gauss(
    Bn * Dn,
    Bn, B_max, 0,
    Dn, D, D_err
)

lam_yellow2 = 579.07 * _nano

A_theo, A_theo_err = err_gauss(
    lam1 / (lam1 - lam2),
    lam2, lam_yellow, 0,
    lam1, lam_yellow2, 0
)

print(f'experimentelles Auflösungsvermögen: Aexp = {A} +- {A_err}')
print(f'maximales Auflösungsvermögen: Amax = {A_max} +- {A_max_err}')
print(f'theoretisches Auflösungsvermögen: Atheo = {A_theo} +- {A_theo_err}')

experimentelles Auflösungsvermögen: Aexp = 353.037776143734 +- 187.230005494681
maximales Auflösungsvermögen: Amax = 3277.26222491567 +- 1738.03150443409
theoretisches Auflösungsvermögen: Atheo = 17.5475757575757 +- 0


In [12]:
# Berechnung Gitterkonstante
gitter_angels = np.array([20+10/30, 19, 15+10/30, 30+21/30, 30, 26 + 12/30]) / _degree
gitter_angel_err = 0.05 / _degree
gitter_lambda = np.array([576.96, 546.07, 407.78, 576.96, 546.07, 407.78]) * _nano
no_angels = np.array([1, 1, 1, 2, 2, 2])

alfa, S, a = sy.symbols('alfa, S, a')

gitterkonstanten = [err_gauss(
    no_angels[i] * gitter_lambda[i] / sy.sin(alfa),
    alfa, gitter_angels[i], gitter_angel_err,
)[0] for i in range(len(no_angels))]

gitterkonstanten_err = [err_gauss(
    no_angels[i] * gitter_lambda[i] / sy.sin(alfa),
    alfa, gitter_angels[i], gitter_angel_err,
)[1] for i in range(len(no_angels))]

gitterkonstante_gew_mittel = gew_mittel(gitterkonstanten, gitterkonstanten_err)

gew. Mittelwert: 2.04655536345380E-7 +/- sigma: 7.84743789802334E-8


In [13]:
# Anzahl beleuchteter Spalte
gitter_N = err_gauss(
    S*a,
    S, 0.41*_milli, 0.005*_milli*np.sqrt(2),
    a, gitterkonstante_gew_mittel[0], gitterkonstante_gew_mittel[1],
)

gitter_N_max = err_gauss(
    1.9*_centi*a,
    a, gitterkonstante_gew_mittel[0], gitterkonstante_gew_mittel[1],
)

gitter_N, gitter_N_max

((8.39087699016059e-11, 3.22070232636838e-11),
 (3.88845519056223e-9, 1.49101320062443e-9))